In [30]:
'Converting our RGB JPEG images from Stylgan to 8 bit gray scale for NBIS'

import os 
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [17]:
'''Putting all the live samples in one folder for TF record'''
data_pd = pd.read_pickle('/home/citer/Desktop/GANS/cm_index_df_fixed.p')
data_pd['FilePath']= data_pd.FilePath.apply(lambda x: os.path.join('/home/citer/Desktop/GANS/Generated_images_512_JPEG/Crossmatch',x))
data_live = data_pd[data_pd.live_spoof == 0] # only live samples
data_live['Collection'] = data_live.CaptureID.apply(lambda x: x[0:3])
resize_df = data_live[data_live.Collection =='Pre']
resize_df.shape

/home/citer/anaconda3/envs/Clarkson_NNI/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


(236345, 21)

In [18]:
unique_subjects = sorted(resize_df.SubjectID.unique())

In [19]:
'''Selecting 250 subjects'''
train_subject = unique_subjects[0:250]

In [20]:
Train_df = pd.DataFrame()
for subject in train_subject:
    sub_df = resize_df[resize_df.SubjectID== subject]
    Train_df = pd.concat([Train_df,sub_df])

In [13]:
Train_df.shape

(72743, 21)

In [7]:
Train_df.to_pickle('/home/citer/Desktop/Fingerprint_synthesize/train_df.p')

In [4]:
Train_df = pd.read_pickle('/home/citer/Desktop/Fingerprint_synthesize/Quality/train_df.p')

In [21]:
Train_df.head()

,CaptureID,FingerType,SubjectID,FilePath,Attempt,Finger,ScannerID,CaptureID.1,live_spoof,sensor_model,...,set_type_fold_1,set_type_fold_2,set_type_fold_3,set_type_fold_4,set_type_fold_5,set_type_fold_6,set_type_fold_7,set_type_fold_8,set_type_fold_9,Collection
95275,Pre_51351,Live,121651651511,/home/citer/Desktop/GANS/Generated_images_512_...,1,1,NaN,Pre_51351,0,crossmatch,...,tr,tr,tr,te,tr,tr,tr,v,tr,Pre
95276,Pre_51352,Live,121651651511,/home/citer/Desktop/GANS/Generated_images_512_...,1,2,NaN,Pre_51352,0,crossmatch,...,tr,tr,tr,te,tr,tr,tr,v,tr,Pre
95277,Pre_51353,Live,121651651511,/home/citer/Desktop/GANS/Generated_images_512_...,1,3,NaN,Pre_51353,0,crossmatch,...,tr,tr,tr,te,tr,tr,tr,v,tr,Pre
95278,Pre_51354,Live,121651651511,/home/citer/Desktop/GANS/Generated_images_512_...,1,4,NaN,Pre_51354,0,crossmatch,...,tr,tr,tr,te,tr,tr,tr,v,tr,Pre
95279,Pre_51355,Live,121651651511,/home/citer/Desktop/GANS/Generated_images_512_...,1,5,NaN,Pre_51355,0,crossmatch,...,tr,tr,tr,te,tr,tr,tr,v,tr,Pre


In [22]:
output_folder_JPEG = '/home/citer/Desktop/GANS/Live_512_JPEG/gray'
#output_folder_BMP = '/home/citer/Desktop/GANS/Live_512_BMP/gray'
if not os.path.exists(output_folder_JPEG):
    os.makedirs(output_folder_JPEG)
#if not os.path.exists(output_folder_BMP):
#    os.makedirs(output_folder_BMP)
    
for i in range(len(Train_df)):
    file = Train_df.iloc[i,3]
    name = file.split('/')[-1]
    arr = cv2.imread(file)
    gray = cv2.cvtColor(arr, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(os.path.join(output_folder_JPEG,name[:-3]+'JPEG'), gray, [int(cv2.IMWRITE_JPEG_QUALITY), 100])
    #cv2.imwrite(os.path.join(output_folder_BMP,name[:-3]+'BMP'), gray)
    os.remove(file)

In [59]:
'''I need to change their size to 512by 512 before minutea extraction'''

input_folder_JPEG = '/home/citer/Desktop/GANS/Live_512_JPEG/gray_400'
output_folder_JPEG = '/home/citer/Desktop/GANS/Live_512_JPEG/gray'

files = os.listdir(input_folder_JPEG)
for file in files:
    arr = cv2.imread(os.path.join(input_folder_JPEG,file))
    gray = cv2.cvtColor(arr, cv2.COLOR_BGR2GRAY)
    image_pad = np.pad(gray, (56, 56), 'constant', constant_values=(255))
    cv2.imwrite(os.path.join(output_folder_JPEG,file), image_pad, [int(cv2.IMWRITE_JPEG_QUALITY), 100])

In [62]:
len(os.listdir(output_folder_JPEG))

72743

In [ ]:
'afterward go to docker and run the Minutea extraction python code'